## Implementing Collaborative Filtering from scratch

### Objective:
The goal of this exercise is to guide you through the implementation of collaborative filtering using matrix factorization and stochastic gradient descent (SGD). By the end of this exercise, students will have a working collaborative filtering system that predicts user ratings for movies

### Step 1: Understanding Collaborative Filtering:
Description: Collaborative filtering is a technique used in recommendation systems that predicts user preferences based on past interactions. It relies on the idea that users with similar preferences in the past will likely have similar preferences in the future.

Task: Research the concept of collaborative filtering and explain the difference between user-based and item-based filtering.

Question: What are the advantages of matrix factorization in collaborative filtering?

Matrix factorization makes it easier to handle large datasets and is more compact in a recommendation structure. It finds latent structure in the data rather than individually scanning everything, which improves processing times. 

### Step 2: Loading the Dataset

Description: The MovieLens dataset contains user ratings for different movies. We will use two files:
* ratings.csv - contains userId, movieId, and rating.
* movies.csv - contains movieId and title.

Task: Load the dataset into a Pandas DataFrame named ratings and movies and inspect the first few rows.

Hint: Use pd.read_csv() to read the files rating.csv and movies.csv.

Question: What are the key columns in the ratings.csv file, and why are they important?

Task: define three variables:
* ratings_shape is the ratings.csv shape, 
* n_users is the number of unique users and 
* n_movies is the number unique movies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Write your code here ro read the ratings and movies datasets:

ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

print(ratings.head())
print()
print(movies.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


The key columns are userId, movieId, and rating. UserId is used to make recommendations for the specific person; movieId is used to categorize movies in order to find others the user may like; and rating is used to judge whether or not the user liked the movie they watched in order to make more accurate recommendations.

### Step 3: Creating the User-Item Matrix

Description: We need to convert our dataset into a user-item matrix where rows represent users, columns represent movies, and values represent ratings.

Task: Use Pandas pivot() function to transform the ratings data. Fill the empty spaces (or nan) with 0.

Hine: type the code in the next cell: user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

The Pandas pivot, will convert the dataframe into another dataframe where the row index is 'userId'. the column index is 'movieId' and the values of the dataframe is 'rating'. 
Think of the result dataframe as the matrix called user_item_matrix.

In [2]:
# Write your code here:

user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

print(user_item_matrix)

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         2.5     0.0     0.0     0.0     0.0     0.0     2.5     0.0   
607         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
608         2.5     2.0     2.0     0.0     0.0     0.0     0.0     0.0   
609         3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
610         5.0     0.0     0.0     0.0     0.0     5.0     0.0     0.0   

movieId  9       10     

### Step 4: Initializing Parameters

Description: We need to initialize two matrices: U (user features) and M (movie features). These matrices represent latent (means hidden) factors that define user preferences and movie characteristics.

Task: Initialize U and M matrices randomly.

Remember: 
* n_users is the number of user
* n_movies is the number of movies
* the latent factors are parameter you can play with, set it to 10 (latents_factors=10)

The latent_factors variable determines the number of hidden features in the matrix factorization model. It represents the dimensions of user preferences and movie characteristics.

Effects:

* Too few factors → The model may not capture enough information, leading to poor recommendations.
* Too many factors → The model may overfit, capturing noise instead of meaningful patterns.

A reasonable number is chosen based on dataset size and experimentation.

In [3]:
# Convert to numpy array
R = user_item_matrix.values
num_users, num_items = R.shape

print('R.shape=', R.shape, 'num_users=', num_users, 'num_movies=', num_items)

# Randomly initialize user and item matrices
latent_factors = 10  # Number of latent features

# Write your code here to define U abd M matrices:
U = np.random.rand(num_users, latent_factors)
M = np.random.rand(num_items, latent_factors)

R.shape= (610, 9724) num_users= 610 num_movies= 9724


### Step 5: Implementing the Training Algorithm

Description: We will use Stochastic Gradient Descent (SGD) to update U and M based on observed ratings. The goal is to minimize the error between predicted and actual ratings.

The dot product between U (user preferences) and M (movie characteristics) represents the predicted rating a user would give to a movie. A higher value indicates a stronger preference, guiding the recommendation system.

Regularization is a term that we add during updating the matricies U and M to helps prevent overfitting by penalizing large values in U and M. This encourages the model to generalize better to unseen data.
* Without regularization: The model may memorize (overfitting) training data, leading to poor performance on new ratings.
* With regularization: The model balances fitting known ratings while maintaining generalization ability.

Task: Implement the SGD update rule for training the model. Type the code below:

```
Learning_rate = 0.01
reg_param = 0.02 # Reqgularization parameter to prevent overfitting
epochs = 100 # number of iterations

for epoch in range(epochs):
    for i in range(num_users): # loop on all users
        for j in range(num_movies): # loop on all movies
            if user_item_matrix.iloc[i, j] > 0:  # Only consider known ratings
                error = user_item_matrix.iloc[i, j] - np.dot(U[i, :], M[j, :].T) # Multiply matrix U[i,:] (specific user) and M[:, j] (specific movie)
                U[i, :] += learning_rate * (error * M[j, :] - reg_param * U[i, :]) # Update user latent_factors for specific user
                M[j, :] += learning_rate * (error * U[i, :] - reg_param * M[j, :]) # Update movie latent_factors for specific movie
    # compute the root mean square error between the actual user_item_matrix and the predicted matrix:
    mse = np.mean((user_item_matrix.values[user_item_matrix.values > 0] - np.dot(U, M.T)[user_item_matrix.values > 0]) ** 2)
    if (epoch+1)%20 == 0: print(f"Epoch {epoch + 1}, MSE: {mse:.4f}") # print root mean square error every 20 iterations
```
Updating U and M only for known ratings ensures that we learn from actual data rather than making arbitrary updates based on missing values. This prevents incorrect assumptions and improves model accuracy.

In [4]:
# Type the code here, when you run the cell it will some time to finish.

learning_rate = 0.01
reg_param = 0.02 # Reqgularization parameter to prevent overfitting
epochs = 20 # number of iterations

### I had to lower the number of epochs because it took my computer 40 minutes to do 20

for epoch in range(epochs):
    for i in range(num_users): # loop on all users
        for j in range(num_items): # loop on all movies
            if user_item_matrix.iloc[i, j] > 0:  # Only consider known ratings
                error = user_item_matrix.iloc[i, j] - np.dot(U[i, :], M[j, :].T) # Multiply matrix U[i,:] (specific user) and M[:, j] (specific movie)
                U[i, :] += learning_rate * (error * M[j, :] - reg_param * U[i, :]) # Update user latent_factors for specific user
                M[j, :] += learning_rate * (error * U[i, :] - reg_param * M[j, :]) # Update movie latent_factors for specific movie
    # compute the root mean square error between the actual user_item_matrix and the predicted matrix:
    mse = np.mean((user_item_matrix.values[user_item_matrix.values > 0] - np.dot(U, M.T)[user_item_matrix.values > 0]) ** 2)
    if (epoch+1)%5 == 0: print(f"Epoch {epoch + 1}, MSE: {mse:.4f}") # print root mean square error every 5 iterations

Epoch 5, MSE: 0.7326
Epoch 10, MSE: 0.6598
Epoch 15, MSE: 0.5977
Epoch 20, MSE: 0.5445


### Step 6: Making Predictions

Description: Now that we have trained U and M, we can make predictions by computing the dot product of U and V.

Task: Implement a function to predict the rating for a specific user and movie. Type the code below:
```
def predict(user, movie): # predict the rating of a user to a movie.
    user_idx = user - 1  # Adjust index
    movie_idx = movie - 1
    # multiple matrix U and M for that specific user and specific movie
    predicted_rating = np.dot(U[user_idx, :], M[movie_idx, :].T)
    # Find the movie name:
    movie_name = movies.loc[movies['movieId'] == movie, 'title'].values[0]
    # Return the predict rating for that specific user and the movie name
    return predicted_rating, movie_name

# Predict rating for user 1 and movie 10
prediction, movie_name = predict(1, 10)
print(f"Predicted rating for User 1 and Movie '{movie_name}': {prediction:.2f}")
```

In [5]:
# Type the code here to do predictions:

def predict(user, movie): # predict the rating of a user to a movie.
    user_idx = user - 1  # Adjust index
    movie_idx = movie - 1
    # multiple matrix U and M for that specific user and specific movie
    predicted_rating = np.dot(U[user_idx, :], M[movie_idx, :].T)
    # Find the movie name:
    movie_name = movies.loc[movies['movieId'] == movie, 'title'].values[0]
    # Return the predict rating for that specific user and the movie name
    return predicted_rating, movie_name

prediction, movie_name = predict(1, 10)
print(f"Predicted rating for User 1 and Movie '{movie_name}': {prediction:.2f}")

Predicted rating for User 1 and Movie 'GoldenEye (1995)': 4.12


### Step 7: Evaluating the Model

Description: Evaluate the performance of the model using Mean Squared Error (MSE) or other relevant metrics.

Task: Compute the overall error on the dataset.

In [6]:
# Write the code here
# Use the root mean squared error (RMSE) formula.

def compute_loss(R, Rp, mask):
    error = np.where(mask, R-Rp, 0)
    loss = np.sum(error**2)
    return 0.5 * loss

print(compute_loss)

<function compute_loss at 0x0000021D68C7C040>


Generate the top-N recommendations for a specific user. We need to compute the predicted ratings for all movies a user hasn't rated, then return the highest ranked movies.
Type in the following code:
```
def recommend_movies(user, N=5):
    user_idx = user - 1  # Adjust index
    predicted_ratings = np.dot(U[user_idx, :], M.T) # multiply the U for a specific user with all movies M latent_factors
    movie_indices = np.argsort(predicted_ratings)[::-1]  # Sort in descending order

    recommended_movies = []
    for movie_idx in movie_indices:
        movie_id = user_item_matrix.columns[movie_idx] # get the movie id
         if user_item_matrix.iloc[user, movie_idx] != 0:
            movie_name = movies.loc[movies['movieId'] == movie_id, 'title'].values[0] # get the movie name
            recommended_movies.append((movie_name, predicted_ratings[movie_idx]))  # append the movie name with predicted rating
        if len(recommended_movies) >= N: # stop when you collect N movies.
            break
    
    return recommended_movies

# Example: Get top 5 recommendations for User 1
top_movies = recommend_movies(1)
for movie, rating in top_movies:
    print(f"Movie: {movie}, Predicted Rating: {rating:.2f}")
```

In [7]:
# Write your code here

def recommend_movies(user, N=5):
    user_idx = user - 1  # Adjust index
    predicted_ratings = np.dot(U[user_idx, :], M.T) # multiply the U for a specific user with all movies M latent_factors
    movie_indices = np.argsort(predicted_ratings)[::-1]  # Sort in descending order

    recommended_movies = []
    for movie_idx in movie_indices:
        movie_id = user_item_matrix.columns[movie_idx] # get the movie id
        if user_item_matrix.iloc[user, movie_idx] != 0:
            movie_name = movies.loc[movies['movieId'] == movie_id, 'title'].values[0] # get the movie name
            recommended_movies.append((movie_name, predicted_ratings[movie_idx]))  # append the movie name with predicted rating
        if len(recommended_movies) >= N: # stop when you collect N movies.
            break
    
    return recommended_movies

# Example: Get top 5 recommendations for User 1
top_movies = recommend_movies(1)
for movie, rating in top_movies:
    print(f"Movie: {movie}, Predicted Rating: {rating:.2f}")

Movie: Whiplash (2014), Predicted Rating: 5.56
Movie: Shawshank Redemption, The (1994), Predicted Rating: 5.40
Movie: Kill Bill: Vol. 1 (2003), Predicted Rating: 5.39
Movie: Inception (2010), Predicted Rating: 5.38
Movie: Departed, The (2006), Predicted Rating: 5.36


#### Submission Instructions

Answer all conceptual questions.

1. Submit the complete Python implementation with comments explaining each step.
2. Provide a short analysis of your model's performance and potential improvements.

Good luck!